In [1]:
import numpy as np
import pandas as pd 
import altair as alt
import copy
import re 
import ntpath 
from commons import data_processing
from commons.DataProcessors import msfragger
from commons.APICallers import uniprot

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
file = r"C:\Users\graha\Downloads\combined_protein (1).tsv"
msf = msfragger.msf_processor([file])
df = msf.data 
df.head(5)

,protein,protein_id,entry_name,gene,protein_length,organism,protein_existence,description,protein_probability,top_peptide_probability,...,f8_intensity,f1_maxlfq_intensity,f2_maxlfq_intensity,f3_maxlfq_intensity,f4_maxlfq_intensity,f5_maxlfq_intensity,f6_maxlfq_intensity,f7_maxlfq_intensity,f8_maxlfq_intensity,indistinguishable_proteins
0,sp|A0A096LP55|QCR6L_HUMAN,A0A096LP55,QCR6L_HUMAN,UQCRHL,91,Homo sapiens OX=9606,3:Protein inferred from homology,"Cytochrome b-c1 complex subunit 6-like, mitoch...",0.9974,0.9949,...,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,sp|P07919|QCR6_HUMAN
1,sp|A0A0U1RRE5|NBDY_HUMAN,A0A0U1RRE5,NBDY_HUMAN,NBDY,68,Homo sapiens OX=9606,1:Experimental evidence at protein level,Negative regulator of P-body association,0.9720,0.9986,...,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,NaN
2,sp|A0A1B0GUS4|UB2L5_HUMAN,A0A1B0GUS4,UB2L5_HUMAN,UBE2L5,154,Homo sapiens OX=9606,2:Experimental evidence at transcript level,Ubiquitin-conjugating enzyme E2 L5,1.0000,0.9990,...,316194912.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,sp|P68036|UB2L3_HUMAN
3,sp|A0AV96|RBM47_HUMAN,A0AV96,RBM47_HUMAN,RBM47,593,Homo sapiens OX=9606,1:Experimental evidence at protein level,RNA-binding protein 47,0.8780,0.9931,...,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,NaN
4,sp|A0AVT1|UBA6_HUMAN,A0AVT1,UBA6_HUMAN,UBA6,1052,Homo sapiens OX=9606,1:Experimental evidence at protein level,Ubiquitin-like modifier-activating enzyme 6,1.0000,0.9990,...,0.0,0.0,0.0,0.0,0.0,1.148814e+09,267059760.0,0.0,0.0,NaN


In [12]:
int_cols = [c for c in df.columns if re.search('maxlfq', c)]
prot_info = ['protein_id', 'entry_name', 'gene', 'description']
data_small = df[prot_info+int_cols]
d = data_small.melt(prot_info)
has_int = d[d.value>0]
has_int.loc[:, 'log2_int'] = np.log10(has_int.value)
has_int = has_int.sort_values('log2_int', ascending=False)
has_int.loc[:, 'rank'] = [i for i in range(len(has_int))]

C:\Users\graha\AppData\Local\Temp\ipykernel_30456\4106455221.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  has_int.loc[:, 'log2_int'] = np.log10(has_int.value)


In [14]:
alt.Chart(has_int).mark_circle(
    size=5
).encode(
    x=alt.X('rank:Q', title='Rank'),
    y=alt.Y('log2_int:Q', title='Log10(intensity)')
).properties(
    height=600,
    width=200
)

alt.Chart(...)